In [1]:
import pickle
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
OUTPUT_PATH = '../../../output_data/validation/'

In [3]:
SOT_PATH = OUTPUT_PATH + 'source_of_truth/'

In [4]:
DATA_PATH = '../../../input_data/synthetic_examples/input_data/'

In [5]:
os.listdir(SOT_PATH)

['one_line',
 'Test0a_15min_Direct_QQ.csv',
 'Test0b_15min_Direct_QQ_RT.csv',
 'test1',
 'Test1_040116_15min_Direct_QQ.csv',
 'Test1_040116_30min_Direct_QQ.csv',
 'Test1_040116_60min_Direct_QQ (2).csv',
 'Test1_040116_90min_Direct_QQ.csv',
 'test2',
 'Test2_091215_15min_Direct_QQ.csv',
 'Test2_091215_30min_Direct_QQ.csv',
 'Test2_091215_45min_Direct_QQ.csv',
 'Test2_091215_60min_Direct_QQ.csv',
 'Test2_091215_90min_Direct_QQ.csv',
 'Test3_040116_15min_Direct_QQ.csv',
 'Test3_040116_30min_Direct_QQ.csv',
 'Test3_040116_45min_Direct_QQ.csv',
 'Test3_040116_60min_Direct_QQ.csv',
 'Test3_040116_90min_Direct_QQ.csv',
 'TestParameters_Statistics.xlsx',
 'two_lines']

## Validate One Line: Test0a

In [6]:
os.listdir(OUTPUT_PATH + 'one_line/')

['aa_0-264.pkl',
 'all_nodes.pkl',
 'morning_direct_edges.pkl',
 'morning_nodes.pkl',
 'morning_start_nodes.pkl',
 'morning_transfer_edges.pkl',
 'nearby_stops.csv',
 'nearby_stops.pkl',
 'sa_0-264.pkl']

In [7]:
aa_ol_df = pd.read_pickle(OUTPUT_PATH + 'one_line/aa_0-264.pkl')

In [8]:
# Limit to 15 min.
aa_ol_df = aa_ol_df[aa_ol_df['time_sec'] <= 900]

In [9]:
aa_ol_df.head()

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure
1,12882,12882,0.0,34.782786,32.091212,2015-12-14 07:28:33,34.782786,32.091212,2015-12-14 07:28:33
2,12882,13042,888.0,34.777718,32.062111,2015-12-14 07:43:21,34.782786,32.091212,2015-12-14 07:28:33
3,12882,13127,330.0,34.781148,32.078797,2015-12-14 07:34:03,34.782786,32.091212,2015-12-14 07:28:33
4,12882,13226,597.0,34.785356,32.069337,2015-12-14 07:38:30,34.782786,32.091212,2015-12-14 07:28:33
9,12882,13362,790.0,34.780759,32.064067,2015-12-14 07:41:43,34.782786,32.091212,2015-12-14 07:28:33


In [10]:
aa_ol_df.shape

(436, 9)

In [11]:
sot_ol_df = pd.read_csv(SOT_PATH + 'one_line/Test0a_15min_Direct_QQ.csv')

In [12]:
sot_ol_df.head()

,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME
0,13948,07:25:00,13948,07:25:00,0
1,13948,07:25:00,12936,07:27:00,120
2,13948,07:25:00,13049,07:27:57,177
3,13948,07:25:00,13100,07:28:53,233
4,13948,07:25:00,14349,07:29:55,295


In [13]:
sot_ol_df.shape

(517, 5)

In [14]:
sot_ol_min_df = sot_ol_df.loc[sot_ol_df.groupby(['ID STOP START', 'ID STOP DEST'])['TRAVEL TIME'].idxmin()]

In [15]:
sot_ol_min_df.shape

(517, 5)

In [16]:
ol_df = aa_ol_df.merge(sot_ol_df, left_on=['stop_id_source', 'stop_id_target'], right_on=['ID STOP START', 'ID STOP DEST'])

In [17]:
ol_df['diff'] = ol_df['time_sec'] - ol_df['TRAVEL TIME']

In [18]:
ol_df.head()

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME,diff
0,12882,12882,0.0,34.782786,32.091212,2015-12-14 07:28:33,34.782786,32.091212,2015-12-14 07:28:33,12882,07:28:33,12882,07:28:33,0,0.0
1,12882,13042,888.0,34.777718,32.062111,2015-12-14 07:43:21,34.782786,32.091212,2015-12-14 07:28:33,12882,07:28:33,13042,07:43:21,888,0.0
2,12882,13127,330.0,34.781148,32.078797,2015-12-14 07:34:03,34.782786,32.091212,2015-12-14 07:28:33,12882,07:28:33,13127,07:34:03,330,0.0
3,12882,13226,597.0,34.785356,32.069337,2015-12-14 07:38:30,34.782786,32.091212,2015-12-14 07:28:33,12882,07:28:33,13226,07:38:30,597,0.0
4,12882,13362,790.0,34.780759,32.064067,2015-12-14 07:41:43,34.782786,32.091212,2015-12-14 07:28:33,12882,07:28:33,13362,07:41:43,790,0.0


In [19]:
ol_df[ol_df['diff'] != 0]

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME,diff


In [20]:
same_stop_df = sot_ol_min_df[sot_ol_min_df['ID STOP START'] == sot_ol_min_df['ID STOP DEST']]

In [21]:
same_stop_df.shape

(42, 5)

In [22]:
sot_ol_diff_df =  sot_ol_min_df[sot_ol_min_df['ID STOP START'] != sot_ol_min_df['ID STOP DEST']]

In [23]:
sot_ol_diff_df.shape

(475, 5)

In [24]:
outer_df = aa_ol_df.merge(sot_ol_diff_df, left_on=['stop_id_source', 'stop_id_target'], right_on=['ID STOP START', 'ID STOP DEST'], how='outer')

In [25]:
outer_df

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME
0,12882.0,12882.0,0.0,34.782786,32.091212,2015-12-14 07:28:33,34.782786,32.091212,2015-12-14 07:28:33,NaN,NaN,NaN,NaN,NaN
1,12882.0,13042.0,888.0,34.777718,32.062111,2015-12-14 07:43:21,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,13042.0,07:43:21,888.0
2,12882.0,13127.0,330.0,34.781148,32.078797,2015-12-14 07:34:03,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,13127.0,07:34:03,330.0
3,12882.0,13226.0,597.0,34.785356,32.069337,2015-12-14 07:38:30,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,13226.0,07:38:30,597.0
4,12882.0,13362.0,790.0,34.780759,32.064067,2015-12-14 07:41:43,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,13362.0,07:41:43,790.0
5,12882.0,13598.0,364.0,34.781313,32.076684,2015-12-14 07:34:37,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,13598.0,07:34:37,364.0
6,12882.0,13679.0,214.0,34.781145,32.082462,2015-12-14 07:32:07,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,13679.0,07:32:07,214.0
7,12882.0,13680.0,109.0,34.782004,32.087089,2015-12-14 07:30:22,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,13680.0,07:30:22,109.0
8,12882.0,14530.0,467.0,34.781736,32.071828,2015-12-14 07:36:20,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,14530.0,07:36:20,467.0
9,12882.0,29488.0,400.0,34.781570,32.075101,2015-12-14 07:35:13,34.782786,32.091212,2015-12-14 07:28:33,12882.0,07:28:33,29488.0,07:35:13,400.0


In [26]:
trips_not_in_my_computations_df = outer_df[outer_df['departure'].isna()]

In [27]:
trips_not_in_my_computations_df.shape

(136, 14)

In [33]:
start_stops = list(trips_not_in_my_computations_df['ID STOP START'].unique())

In [41]:
trips_not_in_my_computations_df[['ID STOP START', 'ID STOP DEST', 'TIME DEPART', 'TIME ARRIVAL', 'TRAVEL TIME']][trips_not_in_my_computations_df['TRAVEL TIME'] <= 900]

,ID STOP START,ID STOP DEST,TIME DEPART,TIME ARRIVAL,TRAVEL TIME


<br><br>
**No problem here, since this is supposed to be limited to 15 minutes (but source of truth didn't limit)**